The following work will implement the following:
1) given a stock ticker, we will retrieve latest news of the company (news agent)
2) given the news, the sentiment agent will evaluate the linked sentiment to the news
3) given the sentiment, a final agent will emit a recommendation to buy, sell or hold the ticker.

In [1]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

In [2]:
from typing import List, Union
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [6]:
type(os.environ["OPENAI_API_KEY"])

str

In [ ]:
# Define our tools
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events, stock news, or stock prices"
    )
]

# Create the prompt template
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template="""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}""",
    tools=tools,
)

# Define the LLM
llm = OpenAI(temperature=0)

# Define the agent
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=None,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Main function
def analyze_stock(ticker):
    query = f"Latest news about {ticker} stock"
    result = agent_executor.run(query)
    return result

# Test the function
if __name__ == "__main__":
    ticker = input("Enter a stock ticker symbol: ")
    analysis = analyze_stock(ticker)
    print(analysis)